In [0]:
-- ===== MONTHLY DENIAL RATE BY PAYER =====

CREATE OR REPLACE VIEW gold.v_monthly_denial_rate AS
SELECT
  date_trunc('month', submission_date)  AS month,
  payer_name,
  COUNT(*)                              AS submitted_claims,
  SUM(has_denial_any)                   AS claims_with_denial,
  SUM(CASE WHEN current_balance <= 0 THEN 1 ELSE 0 END) AS paid_closed_claims,
  SUM(has_denial_any) / COUNT(*)        AS denial_rate
FROM gold.fact_claim
GROUP BY 1, 2;

In [0]:
-- ===== OPEN AR BY PAYER =====

CREATE OR REPLACE VIEW gold.v_aging_buckets AS
SELECT
  payer_name,
  CASE
    WHEN current_balance <= 0 THEN 'Closed'
    WHEN DATEDIFF(current_date(), submission_date) <= 30  THEN '0–30'
    WHEN DATEDIFF(current_date(), submission_date) <= 60  THEN '31–60'
    WHEN DATEDIFF(current_date(), submission_date) <= 90  THEN '61–90'
    WHEN DATEDIFF(current_date(), submission_date) <= 120 THEN '91–120'
    ELSE '120+'
  END AS aging_bucket,
  SUM(current_balance) AS ar_amount,
  COUNT(*)             AS claim_count
FROM gold.fact_claim
GROUP BY 1, 2;

In [0]:
-- ===== TOP DENIAL CATEGORIES ($ IMPACT PROXY) =====

CREATE OR REPLACE VIEW gold.v_top_denial_categories AS
SELECT
  payer_name,
  COALESCE(dominant_denial_category,'Unmapped') AS denial_category,
  COUNT(*)                                      AS claims,
  SUM(billed_amount - net_paid_to_date)         AS gross_delta_amount
FROM gold.fact_claim
WHERE has_denial_any = 1
GROUP BY 1, 2
ORDER BY gross_delta_amount DESC;

In [0]:
-- ===== FIRST-PASS YIELD =====

CREATE OR REPLACE VIEW gold.v_first_pass_yield AS
SELECT
  payer_name,
  SUM(CASE WHEN current_balance <= 0 AND has_denial_any = 0 THEN 1 ELSE 0 END) AS fpy_claims,
  COUNT(*) AS submitted_claims,
  SUM(CASE WHEN current_balance <= 0 AND has_denial_any = 0 THEN 1 ELSE 0 END) / COUNT(*) AS fpy_rate
FROM gold.fact_claim
GROUP BY 1;